Marine Phosphate Cycling and primary productivity
=================================================



## Notes:



-   This `Jupyter Notebook` contains text cells, and code cells.
-   You can edit the text cells using markdown syntax. Just double click this cell to try it out. When finished, press shift+enter.
-   You can execute the python code in each text cell by placing the cursor into a code-cell, and press shift+enter.
-   If you have more then one code cell, you need to execute them in sequential
    order since each cell depends on the output of the previous cell.



## Model structure



In the following, we use a two box ocean model. The surface box represents the `photic zone`  of the ocean, and the deep-box represents the `deep ocean` that is below the photic zone.  The mode approximates the `thermohaline circulation` by two fluxes, $F_u$ which describes the upwelling of deep water, and $F_d$ which describes the downwelling of surface water.

Marine primary productivity (i.e., photosynthesis) is typically limited by the availability of dissolved phosphate. In order to keep the model simple, we assume a linear relationship between the Phosphate concentration and the amount of `organic matter` that is produced by photosynthesis. Since we are only interested in Phosphate, there are no terms for organic matter (OM) production. However, since OM will eventually sink into the deep ocean, the organic bound Phosphate will also sink into the deep ocean. I will call this settling flux $F_{POP}$ which stands for "Particulate Organic Phosphate".  

This flux will depend on the concentration of phosphate, and on the speed how fast organisms can metabolize phosphate (aka phosphate uptake efficiency). There a different ways to describe the uptake efficiency. We could, e.g., state that 70% of the phosphate that is supplied through upwelling ($F_u$) is used for organic matter production. Another way is to state that all phosphate reaching the surface ocean is used within 1 year. In the following we will describe the uptake efficiency as a function of the phosphate residency time in the surface ocean. So $F_{POP}$ is being proportional to the PO<sub>4</sub> concentration and the uptake efficiency $V_S/\tau$:

\begin{equation}\label{}
F_{POP} = [PO_4]_S \times \frac{V_S}{\tau}
\end{equation}

where $\tau$ is the residence of phosphate in the surface ocean and $V_S$ is the volume of the surface box.

We can now describe the equations that govern [PO<sub>4</sub>] in the surface and deep ocean (

![img](./images/mpc.png "F<sub>u</sub> = upwelling, F<sub>d</sub> = downwelling, F<sub>POP</sub> = particulate phosphor, i.e., P that is bound in organic matter. F<sub>w</sub> = weathering, and F<sub>b</sub> = burial")

Similar to the previous  model, our fluxes are not static (with the exception of the weathering flux). $F_u$ and $F_d$ depend on the velocity of thermohaline circulation, and the respective phosphate concentration in each box:

\begin{equation}\label{}
F_d = thc \times [PO_4]
\end{equation}

where `thc` denotes the velocity of  the thermohaline circulation, and $[PO_4]_D$ the concentration in the deep ocean.  A similar equation can be written for $F_d$. 



#### The burial flux



Most of the sinking organic matter will be `remineralized` in the deep
ocean. For our model, we assume that the remineralization efficiency is
about 99% percent. I.e., only 1% percent of the marine export productivity is
buried in the sediments. We can thus write:

\begin{equation}\label{}
F_b = F_{POP} \times (1 -\epsilon)
\end{equation}

where $\epsilon$ denotes the burial efficiency (i.e., 99% = 0.99). 



### Model Equations



For the PO4 concentration in the surface ocean, we can write:

\begin{equation}\label{}
\frac{[PO_{4}]_{S}}{dt} = F_w + [PO_{4}]_{D} \times thc - [PO_{4}]_{S} \times thc - [PO_{4}]_{S} \times \frac{V_S}{\tau}
\end{equation}

and for the deep ocean:

\begin{equation}\label{}
\frac{[PO_4]_S}{dt} =  [PO_4]_S \times \frac{V_S}{\tau} \times \epsilon + [PO_4]_{S} \times thc - [PO_4]_{D} \times thc
\end{equation}

Note that both equations depend on each other, i.e., we need to know the result from one equation to calculate the result of the other equation (i.e., coupled differential equations).



## Model 1



In the following, we will use  the same boundary conditions as given in [&glover-2011-model-method](&glover-2011-model-method).  The model will report the final concentrations, and the how long it took to reach steady state. The model will print a error if the run-time was to short to achieve steady state. In this case increase the model run time (line 16 below). Conversely, if your model reaches steady state too quickly, you can reduce the run time.



In [1]:
"""
A simple P-cycle model, based on chapter 8 of Modeling Methods for the marine
sciences.

This model uses a 2 box ocean, and the P-export flux depends on the biological
productivity in the surface ocean
"""
import po42

F_r = 1.4e3  # River (weathering) flux of PO4 mol/s
V = [30e15, 1.33e18]  # Volume of surface and deep box m^3 [1, 3]
THC = 25  # Sverdrup
tau = 1  # P residency time in surface ocean in years
uptake_efficiency = None  #
burial = 1  # P-burial in %
C_0 = [0, 0]  # initial P in surface and deep box
run_time = 4e6  # run time in years
plot_dpi = 120  #
figure_name = "po421.png"

# ---------- no user serviceable parts below ---------- #
C_s, C_d = po42.po42(
    run_time,
    C_0,
    V,
    F_r,
    plot_dpi,
    figure_name,
    tau,
    uptake_efficiency,
    burial,
    THC,
)

year2sec = 60 * 60 * 24 * 365
THC_yr = THC * year2sec * 1e6
Fu = C_d[-1] * THC_yr
if tau is None:
    Pex = C_d[-1] * THC_yr * uptake_efficiency / 100
else:
    Pex = C_s[-1] * V[0] / tau
e = (1 - (Fu - Pex) / Fu) * 100
Rt_D = V[1] / THC_yr

print(f"PO4 upwelling = {Fu:.2e} [ mol/year]")
print(f"PO4 used in biosynthesis = {Pex:.2e} [ mol/year]")
print(f"\u03B5 = {e:.0f} [%]")
print(f"Deep ocean PO4 residence time = {Rt_D:.2e} [year]")

### Execute the model



You can then execute the code by clicking once in to the code cell, and then press the shift and enter key at the same time (`shift+enter`).



## Model 2



In [1]:
"""
A simple P-cycle model, based on chapter 8 of Modeling Methods for the marine
sciences.

This model uses a 2 box ocean, and the P-export flux depends on the biological
productivity in the surface ocean
"""
import po42

F_r = 1.4e3  # River (weathering) flux of PO4 mol/s
V = [30e15, 1.33e18]  # Volume of surface and deep box m^3 [1, 3]
THC = 20  # Sverdrup
uptake_efficiency = 98  # %
burial = 1  # P-burial in %
tau = None  # P residency time in surface ocean in years
C_0 = [0, 0]  # initial P in surface and deep box
run_time = 4e6  # run time in years
plot_dpi = 120  #
figure_name = "po422.png"

# ---------- no user serviceable parts below ---------- #
C_s, C_d = po42.po42(
    run_time,
    C_0,
    V,
    F_r,
    plot_dpi,
    figure_name,
    tau,
    uptake_efficiency,
    burial,
    THC,
)

year2sec = 60 * 60 * 24 * 365
THC_yr = THC * year2sec * 1e6
Fu = C_d[-1] * THC_yr
if tau == None:
    Pex = C_d[-1] * THC_yr * uptake_efficiency / 100
else:
    Pex = C_s[-1] * V[0] / tau
e = (1 - (Fu - Pex) / Fu) * 100
Rt_D = V[1] / THC_yr

print(f"PO4 upwelling = {Fu:.2e} [ mol/year]")
print(f"PO4 used in biosynthesis = {Pex:.2e} [ mol/year]")
print(f"\u03B5 = {e:.0f} [%]")
print(f"Deep ocean PO4 residence time = {Rt_D:.2e} [year]")

In [1]:
"""
A simple P-cycle model, based on chapter 8 of Modeling Methods for the marine
sciences.

This model uses a 2 box ocean, and the P-export flux depends on the biological
productivity in the surface ocean
"""
import po42

F_r = 2.8e3  # River (weathering) flux of PO4 mol/s
V = [30e15, 1.33e18]  # Volume of surface and deep box m^3 [1, 3]
THC = 20  # Sverdrup
uptake_efficiency = 98  # %
burial = 1  # P-burial in %
tau = None  # P residency time in surface ocean in years
C_0 = [0.212e-3, 7.18e-3]  # initial P in surface and deep box mol/m^3
run_time = 100  # run time in years
plot_dpi = 120  #
figure_name = "po423.png"

# ---------- no user serviceable parts below ---------- #
C_s, C_d = po42.po42(
    run_time,
    C_0,
    V,
    F_r,
    plot_dpi,
    figure_name,
    tau,
    uptake_efficiency,
    burial,
    THC,
    export=True,
    ss=False,
)

year2sec = 60 * 60 * 24 * 365
THC_yr = THC * year2sec * 1e6
Fu = C_d[-1] * THC_yr
if tau == None:
    Pex = C_d[-1] * THC_yr * uptake_efficiency / 100
else:
    Pex = C_s[-1] * V[0] / tau
e = (1 - (Fu - Pex) / Fu) * 100
Rt_D = V[1] / THC_yr

print(f"deep = {C_d[-1]:.4e}")
print(f"PO4 upwelling = {Fu:.2e} [ mol/year]")
print(f"PO4 used in biosynthesis = {Pex:.2e} [ mol/year]")
print(f"\u03B5 = {e:.0f} [%]")
print(f"Deep ocean PO4 residence time = {Rt_D:.2e} [year]")